In [1]:
import sys
import yaml
import torch
import logging
from pathlib import Path

# Add the path to the directory containing the omnicell package
# Assuming the omnicell package is in the parent directory of your notebook
sys.path.append('..')  # Adjust this path as needed

import yaml
import torch
import logging
from pathlib import Path
from omnicell.config.config import Config, ETLConfig, ModelConfig, DatasplitConfig, EvalConfig, EmbeddingConfig
from omnicell.data.loader import DataLoader
from omnicell.constants import PERT_KEY, GENE_EMBEDDING_KEY, CONTROL_PERT
from train import get_model

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configure paths
MODEL_CONFIG = ModelConfig.from_yaml("/orcd/data/omarabu/001/njwfish/omnicell/configs/models/scot/proportional_scot.yaml")
ETL_CONFIG = ETLConfig(name = "no_preprocessing", log1p = False, drop_unmatched_perts = True)
EMBEDDING_CONFIG = EmbeddingConfig(pert_embedding='GenePT')

SPLIT_CONFIG = DatasplitConfig.from_yaml("/orcd/data/omarabu/001/njwfish/omnicell/configs/splits/repogle_k562_essential_raw/random_splits/rs_accP_k562_ood_ss:ns_20_2_most_pert_0.1/split_0/split_config.yaml")
EVAL_CONFIG = EvalConfig.from_yaml("/orcd/data/omarabu/001/njwfish/omnicell/configs/splits/repogle_k562_essential_raw/random_splits/rs_accP_k562_ood_ss:ns_20_2_most_pert_0.1/split_0/eval_config.yaml")  # Set this if you want to run evaluations

# SPLIT_CONFIG = DatasplitConfig.from_yaml("/orcd/data/omarabu/001/njwfish/omnicell/configs/splits/satija_IFNB_raw/random_splits/acrossC_ood_ss:10/split_A549/split_config.yaml")
# EVAL_CONFIG = EvalConfig.from_yaml("/orcd/data/omarabu/001/njwfish/omnicell/configs/splits/satija_IFNB_raw/random_splits/acrossC_ood_ss:10/split_A549/eval_config.yaml")  # Set this if you want to run evaluations


# Load configurations
config = Config(model_config=MODEL_CONFIG,
                 etl_config=ETL_CONFIG, 
                 datasplit_config=SPLIT_CONFIG, 
                 eval_config=EVAL_CONFIG)


#Alternatively you can initialize the config objects manually as follows:
# etl_config = ETLConfig(name = XXX, log1p = False, drop_unmatched_perts = False, ...)
# model_config = ...
# embedding_config = ...
# datasplit_config = ...
# eval_config = ...
# config = Config(etl_config, model_config, datasplit_config, eval_config)


config.etl_config.pert_embedding = 'bioBERT'
config.etl_config.drop_unmatched_perts = True
# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize data loader and load training data
loader = DataLoader(config)
adata, pert_rep_map = loader.get_training_data()

# Get dimensions and perturbation IDs
input_dim = adata.obsm['embedding'].shape[1]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pert_ids = adata.obs[PERT_KEY].unique()
gene_emb_dim = adata.varm[GENE_EMBEDDING_KEY].shape[1] if GENE_EMBEDDING_KEY in adata.varm else None

print(f"Data loaded:")
print(f"- Number of cells: {adata.shape[0]}")
print(f"- Input dimension: {input_dim}")
print(f"- Number of perturbations: {len(pert_ids)}")


2025-02-04 10:25:22,658 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/repogle_k562_essential_raw.yaml
2025-02-04 10:25:22,660 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/satija_IFNB_raw.yaml
2025-02-04 10:25:22,662 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/adamson_INCOMPLETE.yaml
2025-02-04 10:25:22,664 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/satija_IFNB_HVG.yaml
2025-02-04 10:25:22,665 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/kang.yaml
2025-02-04 10:25:22,667 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/essential_gene_knockouts_raw.yaml
2025-02-04 10:25:22,669 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/satija_IFNG_raw_INCOMPLET

Using device: cpu


2025-02-04 10:25:26,621 - INFO - Loaded unpreprocessed data, # of data points: 310385, # of genes: 8563.
2025-02-04 10:25:26,621 - INFO - Preprocessing training data
2025-02-04 10:25:26,624 - INFO - Using identity features for perturbations
2025-02-04 10:25:26,744 - INFO - Removing observations with perturbations not in the dataset as a column
2025-02-04 10:25:26,930 - INFO - Removed 189 perturbations that were not in the dataset columns and 0 perturbations that did not have an embedding for a total of 189 perturbations removed out of an initial 2058 perturbations
/orcd/data/omarabu/001/njwfish/omnicell/notebooks/../omnicell/data/loader.py:175: ImplicitModificationWarning: Setting element `.obsm['embedding']` of view, initializing view as actual.
  adata.obsm["embedding"] = adata.X.toarray().astype('float32')
2025-02-04 10:26:04,454 - INFO - Doing OOD split


Data loaded:
- Number of cells: 279630
- Input dimension: 8563
- Number of perturbations: 1850


In [3]:
import numpy as np
from omnicell.models.datamodules import get_dataloader
dset, dl = get_dataloader(
    adata, pert_ids=np.array(adata.obs[PERT_KEY].values), pert_map=pert_rep_map, offline=False
)


Creating source indices
Creating target indices
Creating pert indices
Creating source and target dicts
Strata probs [1.85915765e-05 1.85915765e-05 1.85915765e-05 ... 3.62163911e-03
 3.68856878e-03 7.42175735e-03]


In [ ]:
import pickle
import time
from omnicell.models.distribute_shift import (
    sample_pert, 
    get_proportional_weighted_dist
)

batch_size = 512

# Add overall timing
total_start_time = time.time()

for stratum in dset.strata:

    iteration_start_time = time.time()
    source_batch = {} 
    synthetic_counterfactual_batch = {}

    num_ctrls = dset.source[stratum].shape[0]

    for i in range(0, num_ctrls, batch_size):   
        source_batch[stratum] = X_ctrl = dset.source[stratum][i:i+batch_size]
        synthetic_counterfactual_batch[stratum] = {}

        mean_ctrl = X_ctrl.mean(axis=0)
        
        # Time the weighted dist calculation
        dist_start = time.time()
        weighted_dist = get_proportional_weighted_dist(X_ctrl)
        
        dist_time = time.time() - dist_start
        print(f"Weighted dist calculation took: {dist_time:.2f}s")

        for j, pert in enumerate(dset.unique_pert_ids):
            if j % 10 == 0:
                pert_start = time.time()
                print(f"{i} / {num_ctrls}, {j} / {len(dset.unique_pert_ids)}")
            
            X_pert = dset.target[stratum][pert]
            mean_pert = X_pert.mean(axis=0)
            mean_shift = mean_pert - mean_ctrl
            
            # Time the sample_pert call
            preds = sample_pert(
                X_ctrl, 
                weighted_dist, 
                mean_shift, 
                max_rejections=100, 
                num_threads=2
            )
            
            synthetic_counterfactual_batch[stratum][pert] = preds.astype(np.int16)
            
            if (j + 1) % 10 == 0:
                pert_time = time.time() - pert_start
                print(f"Perturbation {j} took: {pert_time:.2f}s")
        
    # Save timing data along with results
    data_dict = {
        'synthetic_counterfactuals': synthetic_counterfactual_batch,
        'source': source_batch,
        'unique_pert_ids': dset.unique_pert_ids,
        'strata': dset.strata,
    }
    break

    # with open(f'/orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/proportional_scot/synthetic_counterfactuals_{i}.pkl', 'wb') as f:
    #     pickle.dump(data_dict, f)

Weighted dist calculation took: 0.02s
0 / 10691, 0 / 1849


Perturbation 9 took: 2.00s
0 / 10691, 10 / 1849
Perturbation 19 took: 1.83s
0 / 10691, 20 / 1849
Perturbation 29 took: 1.93s
0 / 10691, 30 / 1849
Perturbation 39 took: 1.91s
0 / 10691, 40 / 1849
Perturbation 49 took: 1.87s
0 / 10691, 50 / 1849
Perturbation 59 took: 1.97s
0 / 10691, 60 / 1849
Perturbation 69 took: 1.96s
0 / 10691, 70 / 1849
Perturbation 79 took: 1.92s
0 / 10691, 80 / 1849
